## words as the demonstration
api call

In [3]:
from openai import OpenAI
import httpx
import openai
import backoff
from utils.postprocess import post_process

api_key = "sk-mE91TMZY8yikxpif8fBa64F0BaBa4d76BcCdD0Cb13F437D2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content.strip('\n')
    # return response


# log message:
# '''


logs = '''Successfully started service 'sparkExecutorActorSystem' on port 55904.'''

log_list = logs.split('\n')

instruct1 = '''You will be provided with some log messages. You should check if the giving log messages share the same template. If so, abstract variables with `{{placeholders}}` to extract the corresponding template.\nPrint the input log's template delimited by backticks.'''

instruct2 = "You will be provided with a log message delimited by backticks. You must abstract variables with `{{placeholders}}` to extract the corresponding template.\nPrint the input log's template delimited by backticks."

# for log in logs:
#     messages = [
#         {'role': 'system', 'content': instruct},
#         {'role': 'user', 'content': f"{log1}\n{log2}"}
#     ]
#     #
    

responce = get_responce( [
    {'role': 'system', 'content': instruct2},
    {'role': 'user', 'content': '10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1874 time: 0.2161739'}
    ])
print(responce)

```
{{ip_address}} "{{method}} {{endpoint}} HTTP/1.1" status: {{status_code}} len: {{response_length}} time: {{response_time}}
```


In [4]:
from utils.postprocess import post_process
print(post_process(responce))

<*> "<*> <*> HTTP/1.<*>" status: <*> len: <*> time: <*>


## DPP to sample the words from logs

In [3]:
# logs = ['proxy.cse.cuhk.edu.hk:5070 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS',
#         '183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:5070 SOCKS5',
#         '223.167.104.147:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001',
#         'qa.sockets.stackexchange.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061',
#         'mtalk.google.com:5228 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection',
#         'tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.',
#         'tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.',
#         'proxy.cse.cuhk.edu.hk:5070 close, 2933 bytes (2.86 KB) sent, 11721005 bytes (11.1 MB) received, lifetime 02:48',
#         'proxy.cse.cuhk.edu.hk:5070 close, 451 bytes sent, 353 bytes received, lifetime <1 sec']

# prompt1 = '''You will be provided with some varaibles, you must discard the variables that are similar or of the same type.
# print the remaining variables in a list delimited by backticks.
# ['183.62.156.108:22', 'socks.cse.cuhk.edu.hk:5070', 'proxy.cse.cuhk.edu.hk:5070', '403 bytes', '426 bytes', '<1 sec', 'video-hkg3-2.xx.fbcdn.net:443', '58373 bytes (57.0 KB)', '8896991 bytes (8.48 MB)', '02:25', 'tcpconn6.tencent.com:443', '182.254.114.110:80']'''

# prompt = '''You will be provided with some varaibles and a log message delimited by backticks. You must abstract variables with `{{placeholders}}` to extract the corresponding template.
# Print the input log's template delimited by backticks.
# varaibles:
# `['183.62.156.108:22', 'socks.cse.cuhk.edu.hk:5070', 'proxy.cse.cuhk.edu.hk:5070', '403 bytes', '426 bytes', '<1 sec', 'video-hkg3-2.xx.fbcdn.net:443', '58373 bytes (57.0 KB)', '8896991 bytes (8.48 MB)', '02:25', 'tcpconn6.tencent.com:443', '182.254.114.110:80']`
import pandas as pd
import re
import numpy as np

from utils.sample_byword import sample_byword

datasets = ['Thunderbird']
for dataset in datasets:
    df = pd.read_csv(
    f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    print(sample_byword(df, 5))

{'an14 an15 an16 an17 an18 an19 an20 an21 an22 an23 an24 an25 an26 an27 an28 an29 an30 an31 an32 an33 an34 an35 an36 an37 an38 an39 an40 an41 an42 an43 an44 an45 an46 an47 an48 an49 an50 an51 an52 an53 an54 an55 an56 an57 an58 an59 an60 an61 an62 an63 an64 an65 an66 an67 an68 an69 an70 an71 an72 an73 an74 an75 an76 an77 an78 an79 an80 an81 an82 an83 an84 an85 an86 an87 an88 an89 an90 an91 an92 an93 an94 an95 an96 an97 an98 an99 an100 an101 an102 an103 an104 an105 an106 an107 an108 an109 an110 an111 an112 an113 an114 an115 an116 an117 an118 an119 an120 an121 an122 an123 an124 an125 an126 an127 an128', '1831', 'root', '0', 'Thunderbird_B7'}


验证按' '分词有多少词，以及含数字的词占多少

In [ ]:
import re
import pandas as pd
from post_process import correct_single_template
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Android','Hadoop', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# datasets = ['Linux']


def tokenize(log_content, tokenize_pattern=r'[ ,]'):
    words = re.split(tokenize_pattern, log_content)
    for index, word in enumerate(words):
        if word.startswith('/') and len(word) > 1:
            words[index] = ''
        if '=' in word:
            words[index] = word.split('=')[0]
        if re.search(r'\d', word):
            words[index] = ''
    words = [word for word in words if word]   # remove null
    return words


a = 0
b = 0
list = []
list2 = []
k = 0
temp = ['structured', 'templates', 'Content', 'EventTemplate']
tokens = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv")
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    print('-' * 20)
    print(dataset)

    # 验证长度为1-3的tempalte是否含有<*>
    # NOTE: 长度为1时，不用解析，2-3仍然需要解析
    # DS
    # NOTE: 没有template中含有'  '
    # BL+US
    # NOTE: boolen变量主要出现在Android中 null:Mac, Android root:many datasets admin:OpenSSH, Thunderbird
    # BL = ['true', 'false']
    # US = ['null', 'root', 'admin']
    # DG
    # NOTE: 没有template含有纯数字
    #
    tem = []
    for log, template in zip(logs, templates):
        if '/' in log and template not in tem:
            words = tokenize(log)
            tem.append(template)
            print(words)
            print(template)
            print('-' * 20)
            
            # print('-' * 20)
    # 验证按' '分词有多少词，以及含数字的词占多少
    # list2.append(len(list))    
    # total_words = 0
    # words_with_numbers = 0

    # for s in list:
    #     words = s.split()
    #     total_words += len(words)
    #     words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    # a+=total_words
    # b+=words_with_numbers


# print(f"{b} / {a}")

In [ ]:
from Levenshtein import distance


def closest_template(target, templates):
    min_distance = float('inf')
    closest_template = None
    for template in templates:
        d = distance(target, template)
        if d < min_distance:
            min_distance = d
            closest_template = template
    return closest_template

templates = [
    '<*> close, <*> bytes sent, <*> bytes received, lifetime <1',
    'proxy.cse.cuhk.edu.hk:<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1',
    '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>',
    '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> MB) received, lifetime <*>',
]

print(closest_template('proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01', templates))

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(target, lst, n=3):
    vectors = [np.array([ord(c) for c in s]).reshape(1, -1) for s in lst]
    target_vector = np.array([ord(c) for c in target]).reshape(1, -1)
    similarities = cosine_similarity(vectors, target_vector)
    most_similar_indices = np.argsort(similarities, axis=0)[-n:].flatten()[::-1]
    return [lst[i] for i in most_similar_indices]

In [ ]:
import pandas as pd
import re
# read log
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print(dataset)
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    for log,template in zip(logs, templates):    
        if 'kb' in log.lower() and template not in list:
            print(log)
            list.append(template)


list = ['sec', 'KB']

查看logpub中oracle template不合理的部分

In [ ]:
from post_process import correct_single_template
import pandas as pd
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print('+' * 20 + dataset)
    df = pd.read_csv(
        f'dataset\{dataset}\{dataset}_2k.log_templates_corrected.csv')
    templates = df['EventTemplate'].tolist()
    for template in templates:
        if correct_single_template(template) != template:
            print(correct_single_template(template))
            print(template)
            print('-' * 20)

try to conclude the types of variables

In [ ]:
import pandas as pd 
import re

def extract_variables(log, template):
    # 将模板中的 <*> 替换为正则表达式的捕获组 (.*?)
    # 为了避免正则表达式的特殊字符导致的问题，先将模板中除了 <*> 外的其他部分进行转义
    # 然后将 <*> 替换为正则表达式的捕获组
    # 这里假设模板中的 <*> 不紧邻正则特殊字符，如果有，需要更复杂的处理
    pattern_parts = template.split("<*>")
    pattern_parts_escaped = [re.escape(part) for part in pattern_parts]
    regex_pattern = "(.*?)".join(pattern_parts_escaped)
    regex = "^" + regex_pattern + "$"  # 添加开始和结束锚点以确保完整匹配

    matches = re.search(regex, log)
    if matches:
        return matches.groups()
    else:
        return []

def check_variable(variable):
    variable = variable.strip()
    if variable.startswith('/'):
        return 'path'
    if variable.startswith('0x'):
        return 'address'
    if re.match(r'\b-?\d+(\.\d+)?\b|\b0x[0-9a-fA-F]+\b', variable):
        return 'number'
    if re.match( r'^[a-zA-Z]+$', variable):
        return 'word'
    if re.match(  r'\b(?:\d{1,3}\.){3}\d{1,3}(?::\d{1,5})?\b', variable):
        return 'ip'
    else:
        return 'null'

table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')

dict = {}

for dataset in datasets:
    dict[dataset] = {}
    print('-' * 20)
    print(dataset + ':')
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    been_used = []
    for log, template in zip(logs, templates):
        if template not in been_used:
            been_used.append(template)
            variables = extract_variables(log, template)
            for variable in variables:
                type = check_variable(variable)
                if type != 'null':
                    if type not in dict[dataset]:
                        dict[dataset][type] = 1
                    else:
                        dict[dataset][type] += 1
                else:
                    print(variable)

print(dict)

Test the unseen accurray

In [ ]:
from collections import Counter
from evaluator import evaluate
import os
import pandas as pd
from sklearn.metrics import accuracy_score

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
# table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac' # logpub
datasets = table_order.split(' ')
a = 0
b = 0
for dataset in datasets:
    file = '0125_0shot'
    input = f'outputs/parser/{file}/{dataset}.csv'  
    output = f'outputs/unseen/{file}/{dataset}.csv'
    os.makedirs(f'outputs/unseen/{file}', exist_ok=True)
    df = pd.read_csv(input)
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    freq = Counter(templates)
    logs_after = []
    templates_after = []
    unseen_templates = [item for item, count in freq.items() if count == 1]
    for log, template in zip(logs, templates):
        if template in unseen_templates:
            logs_after.append(log)
            templates_after.append(template)
    accuracy_exact_string_matching = accuracy_score(templates_after, logs_after, normalize=False)
    length = len(logs_after)
    a += length
    b += accuracy_exact_string_matching
    dataset = ' ' * (12 - len(dataset)) + dataset
    print('%s: len of unseen log: %.4d, Message-Level Accuracy: %.4f' %(dataset, length ,accuracy_exact_string_matching/length))
    

print(b/a)

# 81.0 71.2

## Sample from cluster

In [2]:
import time
from utils.sample_byword import sample_byword
import pandas as pd
# dataset = 'OpenStack'
# df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
# display(sample_byword(df, 5))
logs = '''10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2686100
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2688880
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2576380
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1583 time: 0.1959479
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1910 time: 0.2753000
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2590120
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1575 time: 0.1929309
10.11.21.135,10.11.10.1 "GET /openstack/2013-10-17/vendor_data.json HTTP/1.1" status: 200 len: 124 time: 0.0010960
10.11.21.136,10.11.10.1 "GET /latest/meta-data/placement/availability-zone HTTP/1.1" status: 200 len: 120 time: 0.2186198
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2716691
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2558119
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2777491
10.11.21.137,10.11.10.1 "GET /openstack/2013-10-17/vendor_data.json HTTP/1.1" status: 200 len: 124 time: 0.2337520
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1759 time: 0.2773421
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1759 time: 0.3940661
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1874 time: 0.1995878
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1910 time: 0.2654040
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2709410
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2501280
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1583 time: 0.1754098
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.4256971
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2617428
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2696431
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2692139
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 211 time: 0.1072969
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2648020
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2826021
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2544250
10.11.21.134,10.11.10.1 "GET /openstack/2013-10-17/vendor_data.json HTTP/1.1" status: 200 len: 124 time: 0.0007591
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2732208
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2498078
10.11.21.132,10.11.10.1 "GET /latest/meta-data/ami-launch-index HTTP/1.1" status: 200 len: 117 time: 0.0007291
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2779088
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1874 time: 0.1882880
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2643161
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2630000
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/95960536-049b-41f6-9049-05fc479b6a7c HTTP/1.1" status: 200 len: 1708 time: 0.1721809
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2646649
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2672610
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1583 time: 0.1969190
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.3656721
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2748618
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2636631
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1583 time: 0.1820800
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2691431
10.11.21.131,10.11.10.1 "GET /openstack/2013-10-17/user_data HTTP/1.1" status: 404 len: 176 time: 0.0015490
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2734950
10.11.21.141,10.11.10.1 "GET /latest/meta-data/ HTTP/1.1" status: 200 len: 328 time: 0.2267661
10.11.21.132,10.11.10.1 "GET /latest/meta-data/local-hostname HTTP/1.1" status: 200 len: 130 time: 0.0007689
10.11.21.135,10.11.10.1 "GET /openstack/2012-08-10/meta_data.json HTTP/1.1" status: 200 len: 264 time: 0.3158371'''

logs_list = logs.split('\n')
templates_list = ['<*> "GET <*>" status: <*> len: <*> time: <*>'] * len(logs_list)

df = pd.DataFrame({'Content': logs_list, 'EventTemplate': templates_list})
t1 = time.time()
sample_byword(df, 5, showLogs=True)
print(time.time() - t1)

10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2686100
10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/95960536-049b-41f6-9049-05fc479b6a7c HTTP/1.1" status: 200 len: 1708 time: 0.1721809
10.11.21.136,10.11.10.1 "GET /latest/meta-data/placement/availability-zone HTTP/1.1" status: 200 len: 120 time: 0.2186198
10.11.21.131,10.11.10.1 "GET /openstack/2013-10-17/user_data HTTP/1.1" status: 404 len: 176 time: 0.0015490
10.11.21.135,10.11.10.1 "GET /openstack/2012-08-10/meta_data.json HTTP/1.1" status: 200 len: 264 time: 0.3158371
0.009488344192504883
